<a href="https://colab.research.google.com/github/SergeiShilkoysky/BigData/blob/main/hw1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача: нахождение суммы элементов массива из
1000000 целых чисел.

In [ ]:
import multiprocessing
import threading
import time
from random import randint
import asyncio

COUNT = 10  # количество потоков, процессов, задач

In [ ]:
def generate_array():
    arr = [randint(1, 100) for i in range(1_000_000)]
    # arr = [19, 25, 73, 17, 11, 14, 15, 18, 2, 23, 25, 27, 30, 34, 60, 555]
    return arr

sum_arr = 0

Синхронный подход:

In [ ]:
def sum_array(arr, num):
    global sum_arr
    for i in arr:
        sum_arr += i
    print(f"Сумма элементов массива ({num}): {sum_arr}")

In [ ]:
%%time
arr = generate_array()
print('\t Старт синхронного метода')
sum_arr = 0
sum_array(arr, 1)

	 Старт синхронного метода
Сумма элементов массива (1): 50520255
CPU times: user 895 ms, sys: 803 µs, total: 895 ms
Wall time: 900 ms


Используем многопоточность, многопроцессорность
и асинхронность.

In [ ]:
async def sum_array_async(arr, num):
    global sum_arr
    for i in arr:
        sum_arr += i
    print(f"Сумма элементов массива ({num}): {sum_arr}")

In [ ]:
def sum_use_threading(arr):
    threads = []
    start_time = time.time()
    # Разбиваем на массив на COUNT частей (по количеству потоков)
    for thread_part in range(COUNT):
        thread_start = int(thread_part * (len(arr) / COUNT))
        thread_end = int((thread_part + 1) * (len(arr) / COUNT))
        t = threading.Thread(target=sum_array(arr[thread_start:thread_end], thread_part + 1), args=(arr,))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    end_time = time.time() - start_time
    print('завершение  потоков')
    print('результат суммирования массива:', sum_arr)
    print(f"Общее время работы при многопоточном подходе: {end_time:.10f} seconds \n")
    return end_time

In [ ]:
%%time
sum_arr = 0

print('\t Старт потоков')
sum_use_threading(arr)

	 Старт потоков
Сумма элементов массива (1): 5056951
Сумма элементов массива (2): 10112409
Сумма элементов массива (3): 15162923
Сумма элементов массива (4): 20218515
Сумма элементов массива (5): 25269775
Сумма элементов массива (6): 30306636
Сумма элементов массива (7): 35372566
Сумма элементов массива (8): 40427804
Сумма элементов массива (9): 45478890
Сумма элементов массива (10): 50520255
завершение  потоков
результат суммирования массива: 50520255
Общее время работы при многопоточном подходе: 0.0931398869 seconds 

CPU times: user 89.7 ms, sys: 4.03 ms, total: 93.7 ms
Wall time: 93.4 ms


0.0931398868560791

In [ ]:
def sum_with_process(arr):
    start_time = time.time()
    processes = []
    for proc_part in range(COUNT):
        proc_start = int(proc_part * (len(arr) / COUNT))
        proc_end = int((proc_part + 1) * (len(arr) / COUNT))
        p = multiprocessing.Process(target=sum_array(arr[proc_start:proc_end], proc_part + 1), args=(arr,))
        processes.append(p)
        p.start()
    for p in processes:
        p.join()
    end_time = time.time() - start_time
    print('завершение  процессов')
    print('результат суммирования массива:', sum_arr)
    print(f"Общее время работы при мультипроцессорном подходе: {end_time:.10f} seconds \n")
    return end_time

In [ ]:
%%time
sum_arr = 0
print('\t Старт процессов')

sum_with_process(arr)

	 Старт процессов
Сумма элементов массива (1): 5056951
Сумма элементов массива (2): 10112409
Сумма элементов массива (3): 15162923
Сумма элементов массива (4): 20218515
Сумма элементов массива (5): 25269775
Сумма элементов массива (6): 30306636
Сумма элементов массива (7): 35372566
Сумма элементов массива (8): 40427804
Сумма элементов массива (9): 45478890
Сумма элементов массива (10): 50520255
завершение  процессов
результат суммирования массива: 50520255
Общее время работы при мультипроцессорном подходе: 0.2336764336 seconds 

CPU times: user 150 ms, sys: 70.3 ms, total: 220 ms
Wall time: 236 ms


0.23367643356323242

In [ ]:
async def sum_with_async(arr):
    start_time = time.time()
    tasks = []
    sum_arr = 0
    for assync_part in range(COUNT):
        assync_start = int(assync_part * (len(arr) / COUNT))
        assync_end = int((assync_part + 1) * (len(arr) / COUNT))
        task = asyncio.ensure_future(sum_array_async(arr[assync_start:assync_end], assync_part + 1))
        tasks.append(task)
    await asyncio.gather(*tasks)
    end_time = time.time() - start_time
    print('завершение  асинхронного метода')
    print('результат суммирования массива:', sum_arr)
    print(f"Общее время работы при асинхронном  подходе: {end_time:.10f} seconds \n")
    return end_time

In [ ]:
from asyncio import get_event_loop

In [ ]:
%%time
arr = generate_array()
print('\t Старт асинхронного метода')

loop = asyncio.get_event_loop()
# loop.run_until_complete(sum_with_async(arr)) # выдает RuntimeError: This event loop is already running. В PyCharm версия 3.12 команда отрабатывает

	 Старт асинхронного метода
CPU times: user 816 ms, sys: 5.55 ms, total: 822 ms
Wall time: 824 ms
